In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('\\')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  c:\RAI\latest\fairness-variance


## Import dependencies

In [4]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import CreditCardDefaultDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_model_params_for_mult_repair_levels
from source.experiment_interface import run_exp_iter_with_eqq_odds_postprocessing

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [5]:
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'postprocessing_intervention'
DB_COLLECTION_NAME = 'eq_odds_postprocessing'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'credit_card_default_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

In [6]:
metrics_computation_config

DefaultMunch(None, {'dataset_name': 'Credit_Card_Default', 'bootstrap_fraction': 0.8, 'n_estimators': 3, 'computation_mode': 'postprocessing_intervention', 'num_runs': 1, 'runs_seed_lst': [100], 'sensitive_attributes_dct': DefaultMunch(None, {'sex': 'female'})})

## Define a db writer and custom fields to insert into your database

In [7]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'nazar_experiments'

In [8]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [9]:
import uuid

custom_table_fields_dct = {
    #'session_uuid': str(uuid.uuid4()),
    'session_uuid': "test"
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  test


## Initialize custom objects

In [10]:
data_loader = CreditCardDefaultDataset()
data_loader.X_data.head()

,limit_bal,age,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,...,pay_amt6,sex,education,marriage,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6
0,20000,24,3913,3102,689,0,0,0,0,689,...,0,female,2,1,2,2,-1,-1,-2,-2
1,120000,26,2682,1725,2682,3272,3455,3261,0,1000,...,2000,female,2,2,-1,2,0,0,0,2
2,90000,34,29239,14027,13559,14331,14948,15549,1518,1500,...,5000,female,2,2,0,0,0,0,0,0
3,50000,37,46990,48233,49291,28314,28959,29547,2000,2019,...,1000,female,2,1,0,0,0,0,0,0
4,50000,57,8617,5670,35835,20940,19146,19131,2000,36681,...,679,male,2,1,-1,0,-1,0,0,0


In [11]:
data_loader.X_data.shape

(29946, 23)

## Run experiment iterations

In [12]:
tuned_params_filenames = [
    'tuning_results_Credit_Card_Default_20231023__200422.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]

### Experiment iteration 1

In [13]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [14]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)

run_exp_iter_with_eqq_odds_postprocessing(data_loader=exp_iter_data_loader,
                                           experiment_seed=experiment_seed,
                                           test_set_fraction=TEST_SET_FRACTION,
                                           db_writer_func=db_writer_func,
                                           models_params_for_tuning=models_params_for_tuning,
                                           metrics_computation_config=metrics_computation_config,
                                           custom_table_fields_dct=custom_table_fields_dct,
                                           with_tuning=False,
                                           tuned_params_df_path=tuned_params_df_paths[0],
                                           save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                           verbose=True,
                                           dataset_name='CreditCardDefaultDataset')

2023-11-06 23:26:41 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[1.0, 0.0]',
 'model_init_seed': 100,
 'session_uuid': 'test'}




2023-11-06 23:26:41 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2023-11-06 23:26:41 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file
2023-11-06 23:26:41 experiment_interface.py INFO    : Start computing metrics with postprocessing
INFO:root:Start computing metrics with postprocessing


Top indexes of an X_test in a base flow dataset:  Int64Index([18038,  8066, 24728,  5110, 11579,  7954, 10526, 17138, 22689,
             3145, 17602, 25196,  3840,  5898, 19331, 29687, 29109,  7278,
            19632, 28566],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([18038,  8066, 24728,  5110, 11579,  7954, 10526, 17138, 22689,
             3145, 17602, 25196,  3840,  5898, 19331, 29687, 29109,  7278,
            19632, 28566],
           dtype='int64')
Path for tuned params:  c:\RAI\latest\fairness-variance\results\postprocessing_intervention\tuning_results_Credit_Card_Default_20231023__200422.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 50, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': None, 'oob_sc

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/3 [00:00<?, ?it/s]

Model computation config without postprocessing:  DefaultMunch(None, {'dataset_name': 'Credit_Card_Default', 'bootstrap_fraction': 0.8, 'n_estimators': 3, 'computation_mode': 'error_analysis', 'num_runs': 1, 'runs_seed_lst': [100], 'sensitive_attributes_dct': DefaultMunch(None, {'sex': 'female'})})


Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
q

NameError: name 'q' is not defined

### Experiment iteration 2

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)

run_exp_iter_with_eqq_odds_postprocessing(data_loader=exp_iter_data_loader,
                                           experiment_seed=experiment_seed,
                                           test_set_fraction=TEST_SET_FRACTION,
                                           db_writer_func=db_writer_func,
                                           models_params_for_tuning=models_params_for_tuning,
                                           metrics_computation_config=metrics_computation_config,
                                           custom_table_fields_dct=custom_table_fields_dct,
                                           with_tuning=False,
                                           tuned_params_df_path=tuned_params_df_paths[0],
                                           save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                           verbose=True,
                                           dataset_name='CreditCardDefaultDataset')

2023-11-01 22:20:43 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[1.0, 0.0]',
 'intervention_param': 0.0,
 'model_init_seed': 200,
 'session_uuid': 'acafb64d-0b5a-4b37-bd3c-bd234d2ab800'}




2023-11-01 22:20:43 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2023-11-01 22:20:43 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file
2023-11-01 22:20:43 experiment_interface.py INFO    : Start computing metrics with postprocessing
INFO:root:Start computing metrics with postprocessing


Top indexes of an X_test in a base flow dataset:  Int64Index([21680, 20224,  5915,  5049, 13870,  3480, 23820, 13216, 13055,
            28384,  9031, 16516, 13182, 21886, 21179, 21466, 24922, 15658,
            12239, 20532],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([21680, 20224,  5915,  5049, 13870,  3480, 23820, 13216, 13055,
            28384,  9031, 16516, 13182, 21886, 21179, 21466, 24922, 15658,
            12239, 20532],
           dtype='int64')
Path for tuned params:  c:\RAI\fairness-variance-latest\fairness-variance\results\eq_odds_postprocessing_credit_card\tuning_results_Credit_Card_Default_20231023__200422.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 50, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

...

dict_keys(['sex_priv', 'sex_dis'])
                  overall  sex_priv   sex_dis
Jitter           0.000447  0.000548  0.000386
Label_Stability  0.999553  0.999452  0.999614
                    overall  sex_priv   sex_dis
TPR                0.341791  0.347328  0.338235
TNR                0.945376  0.937860  0.949863
PPV                0.643258  0.627586  0.654028
FNR                0.658209  0.652672  0.661765
FPR                0.054624  0.062140  0.050137
Accuracy           0.810351  0.801061  0.815987
F1                 0.446394  0.447174  0.445880
Selection-Rate     0.118865  0.128205  0.113197
Positive-Rate      0.531343  0.553435  0.517157
Sample_Size     2995.000000       NaN       NaN
metrics_computation_interfaces.py: model_metrics_df:               Metric      overall  sex_priv   sex_dis              Model_Name  \
0            Jitter     0.000447  0.000548  0.000386  RandomForestClassifier   
1   Label_Stability     0.999553  0.999452  0.999614  RandomForestClassifier   
2    

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

sentitive_attributes_dct:  DefaultMunch(None, {'sex': 'female'})
postprocessing_sensitive_attribute:  None


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

                        overall  sex_priv  sex_priv_correct  \
Mean_Prediction        0.772491  0.756737          0.781114   
Statistical_Bias       0.279231  0.293340          0.183982   
Std                    0.047713  0.049188          0.045570   
Overall_Uncertainty    0.635353  0.661570          0.626707   
Aleatoric_Uncertainty  0.622304  0.648463          0.614766   
IQR                    0.065359  0.067442          0.062264   
Jitter                 0.042226  0.050099          0.033506   
Label_Stability        0.941119  0.929797          0.952492   

                       sex_priv_incorrect   sex_dis  sex_dis_correct  \
Mean_Prediction                  0.662273  0.782050         0.800244   
Statistical_Bias                 0.717099  0.270671         0.167890   
Std                              0.063208  0.046818         0.044121   
Overall_Uncertainty              0.796664  0.619445         0.592854   
Aleatoric_Uncertainty            0.779040  0.606431         0.580631   


### Experiment iteration 3

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)

run_exp_iter_with_eqq_odds_postprocessing(data_loader=exp_iter_data_loader,
                                           experiment_seed=experiment_seed,
                                           test_set_fraction=TEST_SET_FRACTION,
                                           db_writer_func=db_writer_func,
                                           models_params_for_tuning=models_params_for_tuning,
                                           metrics_computation_config=metrics_computation_config,
                                           custom_table_fields_dct=custom_table_fields_dct,
                                           with_tuning=False,
                                           tuned_params_df_path=tuned_params_df_paths[0],
                                           save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                           verbose=True,
                                           dataset_name='CreditCardDefaultDataset')

2023-11-01 23:35:02 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': '[1.0, 0.0]',
 'intervention_param': 0.0,
 'model_init_seed': 300,
 'session_uuid': 'acafb64d-0b5a-4b37-bd3c-bd234d2ab800'}




2023-11-01 23:35:02 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2023-11-01 23:35:02 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file
2023-11-01 23:35:02 experiment_interface.py INFO    : Start computing metrics with postprocessing
INFO:root:Start computing metrics with postprocessing


Top indexes of an X_test in a base flow dataset:  Int64Index([26463, 13134,    73,  8075, 18107,  7878, 21765, 28479,  7222,
            24719,  6544, 15508, 28160,  4396, 14563, 15019, 12935,  7442,
            25921, 23729],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([26463, 13134,    73,  8075, 18107,  7878, 21765, 28479,  7222,
            24719,  6544, 15508, 28160,  4396, 14563, 15019, 12935,  7442,
            25921, 23729],
           dtype='int64')
Path for tuned params:  c:\RAI\fairness-variance-latest\fairness-variance\results\eq_odds_postprocessing_credit_card\tuning_results_Credit_Card_Default_20231023__200422.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 50, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

...

dict_keys(['sex_priv', 'sex_dis'])
                  overall  sex_priv   sex_dis
Jitter           0.000487  0.000589  0.000422
Label_Stability  0.999513  0.999411  0.999578
                    overall  sex_priv   sex_dis
TPR                0.356589  0.385768  0.335979
TNR                0.953191  0.949171  0.955709
PPV                0.676471  0.691275  0.664921
FNR                0.643411  0.614232  0.664021
FPR                0.046809  0.050829  0.044291
Accuracy           0.824708  0.820819  0.827208
F1                 0.467005  0.495192  0.446397
Selection-Rate     0.113523  0.127133  0.104772
Positive-Rate      0.527132  0.558052  0.505291
Sample_Size     2995.000000       NaN       NaN
metrics_computation_interfaces.py: model_metrics_df:               Metric      overall  sex_priv   sex_dis              Model_Name  \
0            Jitter     0.000487  0.000589  0.000422  RandomForestClassifier   
1   Label_Stability     0.999513  0.999411  0.999578  RandomForestClassifier   
2    

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

sentitive_attributes_dct:  DefaultMunch(None, {'sex': 'female'})
postprocessing_sensitive_attribute:  None


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

                        overall  sex_priv  sex_priv_correct  \
Mean_Prediction        0.775937  0.759118          0.777972   
Statistical_Bias       0.269662  0.280242          0.185400   
Std                    0.048270  0.049515          0.046574   
Overall_Uncertainty    0.637118  0.657430          0.630938   
Aleatoric_Uncertainty  0.623677  0.644060          0.618530   
IQR                    0.066113  0.067598          0.063572   
Jitter                 0.042559  0.049973          0.033943   
Label_Stability        0.939750  0.928780          0.951886   

                       sex_priv_incorrect   sex_dis  sex_dis_correct  \
Mean_Prediction                  0.671225  0.786750         0.802845   
Statistical_Bias                 0.722378  0.262861         0.170450   
Std                              0.063228  0.047470         0.044713   
Overall_Uncertainty              0.780929  0.624060         0.598718   
Aleatoric_Uncertainty            0.763074  0.610573         0.586119   


### Experiment iteration 4

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)

run_exp_iter_with_eqq_odds_postprocessing(data_loader=exp_iter_data_loader,
                                           experiment_seed=experiment_seed,
                                           test_set_fraction=TEST_SET_FRACTION,
                                           db_writer_func=db_writer_func,
                                           models_params_for_tuning=models_params_for_tuning,
                                           metrics_computation_config=metrics_computation_config,
                                           custom_table_fields_dct=custom_table_fields_dct,
                                           with_tuning=False,
                                           tuned_params_df_path=tuned_params_df_paths[0],
                                           save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                           verbose=True,
                                           dataset_name='CreditCardDefaultDataset')

2023-11-02 00:50:17 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 400,
 'experiment_iteration': 'Exp_iter_4',
 'fair_intervention_params_lst': '[1.0, 0.0]',
 'intervention_param': 0.0,
 'model_init_seed': 400,
 'session_uuid': 'acafb64d-0b5a-4b37-bd3c-bd234d2ab800'}




2023-11-02 00:50:17 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2023-11-02 00:50:17 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file
2023-11-02 00:50:17 experiment_interface.py INFO    : Start computing metrics with postprocessing
INFO:root:Start computing metrics with postprocessing


Top indexes of an X_test in a base flow dataset:  Int64Index([  419,  9188, 10232, 18924, 29114,  4791,  2589, 15158, 14011,
            18163, 20276, 12033, 28585,  1327, 22167, 10055,  6290, 13984,
            14472, 29620],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([  419,  9188, 10232, 18924, 29114,  4791,  2589, 15158, 14011,
            18163, 20276, 12033, 28585,  1327, 22167, 10055,  6290, 13984,
            14472, 29620],
           dtype='int64')
Path for tuned params:  c:\RAI\fairness-variance-latest\fairness-variance\results\eq_odds_postprocessing_credit_card\tuning_results_Credit_Card_Default_20231023__200422.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 50, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

...

dict_keys(['sex_priv', 'sex_dis'])
                  overall  sex_priv   sex_dis
Jitter           0.000501  0.000485  0.000511
Label_Stability  0.999499  0.999515  0.999489
                    overall  sex_priv   sex_dis
TPR                0.368732  0.342373  0.389034
TNR                0.944756  0.927858  0.955508
PPV                0.661376  0.608434  0.702830
FNR                0.631268  0.657627  0.610966
FPR                0.055244  0.072142  0.044492
Accuracy           0.814357  0.783445  0.834908
F1                 0.473485  0.438178  0.500840
Selection-Rate     0.126210  0.138796  0.117843
Positive-Rate      0.557522  0.562712  0.553525
Sample_Size     2995.000000       NaN       NaN
metrics_computation_interfaces.py: model_metrics_df:               Metric      overall  sex_priv   sex_dis              Model_Name  \
0            Jitter     0.000501  0.000485  0.000511  RandomForestClassifier   
1   Label_Stability     0.999499  0.999515  0.999489  RandomForestClassifier   
2    

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

sentitive_attributes_dct:  DefaultMunch(None, {'sex': 'female'})
postprocessing_sensitive_attribute:  None


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

                        overall  sex_priv  sex_priv_correct  \
Mean_Prediction        0.771644  0.759122          0.783743   
Statistical_Bias       0.274378  0.295975          0.181258   
Std                    0.047794  0.049015          0.045903   
Overall_Uncertainty    0.635682  0.657540          0.621927   
Aleatoric_Uncertainty  0.622627  0.644509          0.609678   
IQR                    0.065690  0.067284          0.063028   
Jitter                 0.044285  0.049119          0.032493   
Label_Stability        0.936952  0.930067          0.953907   

                       sex_priv_incorrect   sex_dis  sex_dis_correct  \
Mean_Prediction                  0.667811  0.779968         0.798409   
Statistical_Bias                 0.721418  0.260020         0.169029   
Std                              0.060556  0.046983         0.043768   
Overall_Uncertainty              0.789615  0.621151         0.593049   
Aleatoric_Uncertainty            0.773685  0.608080         0.581165   


### Experiment iteration 5

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)

run_exp_iter_with_eqq_odds_postprocessing(data_loader=exp_iter_data_loader,
                                           experiment_seed=experiment_seed,
                                           test_set_fraction=TEST_SET_FRACTION,
                                           db_writer_func=db_writer_func,
                                           models_params_for_tuning=models_params_for_tuning,
                                           metrics_computation_config=metrics_computation_config,
                                           custom_table_fields_dct=custom_table_fields_dct,
                                           with_tuning=False,
                                           tuned_params_df_path=tuned_params_df_paths[0],
                                           save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                           verbose=True,
                                           dataset_name='CreditCardDefaultDataset')

2023-11-02 02:02:04 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 500,
 'experiment_iteration': 'Exp_iter_5',
 'fair_intervention_params_lst': '[1.0, 0.0]',
 'intervention_param': 0.0,
 'model_init_seed': 500,
 'session_uuid': 'acafb64d-0b5a-4b37-bd3c-bd234d2ab800'}




2023-11-02 02:02:04 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2023-11-02 02:02:04 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file
2023-11-02 02:02:04 experiment_interface.py INFO    : Start computing metrics with postprocessing
INFO:root:Start computing metrics with postprocessing


Top indexes of an X_test in a base flow dataset:  Int64Index([ 4180, 15081, 21731, 27459, 29788, 24643, 14768, 14421, 20198,
             9786, 10152, 16825, 28442,  2667,  4835, 11415, 20290,  9653,
            26492, 25949],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 4180, 15081, 21731, 27459, 29788, 24643, 14768, 14421, 20198,
             9786, 10152, 16825, 28442,  2667,  4835, 11415, 20290,  9653,
            26492, 25949],
           dtype='int64')
Path for tuned params:  c:\RAI\fairness-variance-latest\fairness-variance\results\eq_odds_postprocessing_credit_card\tuning_results_Credit_Card_Default_20231023__200422.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 50, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

...

dict_keys(['sex_priv', 'sex_dis'])
                  overall  sex_priv   sex_dis
Jitter           0.005947  0.005190  0.006435
Label_Stability  0.988611  0.990153  0.987618
                    overall  sex_priv   sex_dis
TPR                0.391104  0.376344  0.402145
TNR                0.944089  0.935123  0.949620
PPV                0.660622  0.644172  0.672646
FNR                0.608896  0.623656  0.597855
FPR                0.055911  0.064877  0.050380
Accuracy           0.823706  0.802217  0.837541
F1                 0.491329  0.475113  0.503356
Selection-Rate     0.128881  0.138960  0.122393
Positive-Rate      0.592025  0.584229  0.597855
Sample_Size     2995.000000       NaN       NaN
metrics_computation_interfaces.py: model_metrics_df:               Metric      overall  sex_priv   sex_dis              Model_Name  \
0            Jitter     0.005947  0.005190  0.006435  RandomForestClassifier   
1   Label_Stability     0.988611  0.990153  0.987618  RandomForestClassifier   
2    

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

sentitive_attributes_dct:  DefaultMunch(None, {'sex': 'female'})
postprocessing_sensitive_attribute:  None


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

                        overall  sex_priv  sex_priv_correct  \
Mean_Prediction        0.766467  0.751422          0.774896   
Statistical_Bias       0.271332  0.291301          0.188520   
Std                    0.047813  0.048911          0.045962   
Overall_Uncertainty    0.641765  0.670097          0.638103   
Aleatoric_Uncertainty  0.628836  0.657435          0.626188   
IQR                    0.065502  0.067178          0.063001   
Jitter                 0.041280  0.045602          0.031353   
Label_Stability        0.943065  0.937783          0.958051   

                       sex_priv_incorrect   sex_dis  sex_dis_correct  \
Mean_Prediction                  0.654657  0.776153         0.795404   
Statistical_Bias                 0.714989  0.258475         0.170053   
Std                              0.061064  0.047107         0.044391   
Overall_Uncertainty              0.801986  0.623525         0.596894   
Aleatoric_Uncertainty            0.786243  0.610424         0.584602   
